In [29]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 포트폴리오별 수익률 계산

In [30]:
quant = ['peg', 'ncav', 'low']
# fname = '../datasets/portfolio_' + quant[0] + '.csv'
fname = '../datasets/portfolio_' + quant[1] + '.csv'
df_portfolio = pd.read_csv(fname, dtype={'거래소코드':'object'}, index_col=0)
df_portfolio

,year,회사명,거래소코드,회계년도,시가총액(백만원),유동자산(백만원),부채(백만원),당기순이익(백만원),NCAV,NCAV/시총,조건1,조건2,최종조건,pf_ratio
0,2008,(주)동성화학,005190,2007/12,6.766614e+03,136542.0,91447.0,15021.0,45095.0,6.664337,True,True,True,0.006144
1,2008,(주)씨앤우방,013200,2007/12,6.561833e+03,455492.0,423375.0,26.0,32117.0,4.894517,True,True,True,0.005959
2,2008,KISCO홀딩스(주),001940,2007/12,9.876600e+04,629813.0,333068.0,73488.0,296745.0,3.004526,True,True,True,0.089685
3,2008,(주)에스와이코퍼레이션,008080,2007/12,9.438399e+03,55806.0,28970.0,1954.0,26836.0,2.843279,True,True,True,0.008571
4,2008,(주)윌비스,008600,2007/12,2.430670e+04,118460.0,60169.0,5922.0,58291.0,2.398145,True,True,True,0.022072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,2023,태광산업(주),003240,2022/12,6.791740e+05,1928389.0,702185.0,342230.0,1226204.0,1.805434,True,True,True,0.119891
253,2023,(주)대덕,008060,2022/12,2.165581e+05,803619.0,419083.0,133396.0,384536.0,1.775672,True,True,True,0.038228
254,2023,케이피엑스홀딩스(주),092230,2022/12,2.167243e+05,812718.0,448980.0,37511.0,363738.0,1.678344,True,True,True,0.038257
255,2023,(주)유수홀딩스,000700,2022/12,1.507821e+05,325176.0,79255.0,44119.0,245921.0,1.630970,True,True,True,0.026617


In [31]:
# 수익률 계산에 필요한 컬럼만 남기기
df_portfolio = df_portfolio[['year', '거래소코드', 'pf_ratio']]
df_portfolio

,year,거래소코드,pf_ratio
0,2008,005190,0.006144
1,2008,013200,0.005959
2,2008,001940,0.089685
3,2008,008080,0.008571
4,2008,008600,0.022072
...,...,...,...
252,2023,003240,0.119891
253,2023,008060,0.038228
254,2023,092230,0.038257
255,2023,000700,0.026617


In [32]:
# 연도별 포트폴리오 편입 종목 수
df_portfolio['year'].sort_values().value_counts()

year
2008    20
2011    20
2018    20
2019    20
2020    20
2021    20
2022    20
2023    20
2012    19
2017    18
2016    13
2009    12
2010    12
2013     9
2014     7
2015     7
Name: count, dtype: int64

In [33]:
df_kospi = pd.read_csv('../datasets/kospi.csv', index_col=0)
df_kospi.index = pd.to_datetime(df_kospi.index)
df_kospi

,Adj Close,event,귀속년도,rtn
Date,,,,
1996-12-12,689.380005,before,1996,-2.171197
1996-12-13,689.070007,before,1996,-0.044968
1996-12-16,673.919983,before,1996,-2.198619
1996-12-17,663.349976,before,1996,-1.568437
1996-12-18,661.390015,before,1996,-0.295464
...,...,...,...,...
2024-01-02,2669.810059,santa,2023,0.547213
2024-01-03,2607.310059,santa,2023,-2.340991
2024-01-04,2587.020020,after,2023,-0.778198


In [34]:
# 연도별 수익률 계산
import yfinance as yf
from pykrx import stock
from tqdm import tqdm

years = sorted(df_portfolio['year'].unique())

portfolio_datas = {}
for year in years:
    print(year, end=' : ')
    portfolio_datas[year] = {}
    
    portfolio = df_portfolio[df_portfolio['year']==year]
    portfolio_datas[year]['portfolio'] = portfolio
    
    kospi = df_kospi[(df_kospi['귀속년도']==year)]
    
    # else 마지막날 --> before 첫날 수익률 계산 목적
    from_date = kospi[kospi['event']=='else'].index[-1]
    print(from_date)

    # after 첫날 = santa 다음날 (yfinance는 end 전날까지 보여줌)
    to_date = kospi[kospi['event']=='after'].index[0]
    print(to_date)
    
    kospi = kospi.loc[from_date:to_date]
    
    rtns = []
    tickers_pykrx = []
    print(f'{from_date} ~ {to_date}')
    for ticker in tqdm(portfolio['거래소코드']):
        
        rtn = yf.download(
            ticker+'.KS', start = from_date, end=to_date,
            progress=False
        )[['Adj Close']]
        
        if rtn.empty:
            # yfinance에서 다운로드 실패 -> pykrx
            tickers_pykrx.append(ticker)
            rtn = stock.get_market_ohlcv(fromdate=from_date, todate=to_date, ticker=ticker)[['종가', '등락률']]
            rtn['rtn'] = rtn['등락률'] / 100
            # pass
        else:
            rtn['rtn'] = rtn['Adj Close'].pct_change() # Adj Close 기준 일별 수익률 계산
            
            # rtn.columns = ['price_'+ticker, 'rtn_'+ticker]
            rtn = rtn[['rtn']]
            rtn.columns = [ticker]
            rtns.append(rtn.dropna())
    print(f'{from_date} ~ {to_date} {len(rtns)}')
    
    rtns = pd.concat(rtns, axis=1)
    portfolio_datas[year]['rtns'] = rtns
    portfolio_datas[year]['tickers_pykrx'] = tickers_pykrx

2008 : 2008-12-11 00:00:00
2009-01-06 00:00:00
2008-12-11 00:00:00 ~ 2009-01-06 00:00:00


  0%|          | 0/20 [00:00<?, ?it/s]


1 Failed download:
['005190.KS']: Exception('%ticker%: No timezone found, symbol may be delisted')
  5%|▌         | 1/20 [00:03<01:01,  3.22s/it]
1 Failed download:
['013200.KS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2008-12-11 00:00:00 -> 2009-01-06 00:00:00)')
 25%|██▌       | 5/20 [00:04<00:09,  1.59it/s]
1 Failed download:
['001600.KS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2008-12-11 00:00:00 -> 2009-01-06 00:00:00)')
 30%|███       | 6/20 [00:05<00:09,  1.51it/s]
1 Failed download:
['014350.KS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2008-12-11 00:00:00 -> 2009-01-06 00:00:00)')
100%|██████████| 20/20 [00:11<00:00,  1.80it/s]


2008-12-11 00:00:00 ~ 2009-01-06 00:00:00 16
2009 : 2009-12-11 00:00:00
2010-01-06 00:00:00
2009-12-11 00:00:00 ~ 2010-01-06 00:00:00


  0%|          | 0/12 [00:00<?, ?it/s]
1 Failed download:
['950010.KS']: Exception('%ticker%: No timezone found, symbol may be delisted')
 25%|██▌       | 3/12 [00:02<00:06,  1.47it/s]
1 Failed download:
['014350.KS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2009-12-11 00:00:00 -> 2010-01-06 00:00:00)')
100%|██████████| 12/12 [00:06<00:00,  1.91it/s]


...

2021-12-14 00:00:00 ~ 2022-01-06 00:00:00 20
2022 : 2022-12-13 00:00:00
2023-01-04 00:00:00
2022-12-13 00:00:00 ~ 2023-01-04 00:00:00


100%|██████████| 20/20 [00:07<00:00,  2.55it/s]


2022-12-13 00:00:00 ~ 2023-01-04 00:00:00 20
2023 : 2023-12-11 00:00:00
2024-01-04 00:00:00
2023-12-11 00:00:00 ~ 2024-01-04 00:00:00


100%|██████████| 20/20 [00:07<00:00,  2.66it/s]

2023-12-11 00:00:00 ~ 2024-01-04 00:00:00 20


In [35]:
# tickers_pykrx --> 산타랠리 기간에 상장폐지된 종목 ??
df = pd.DataFrame(portfolio_datas).transpose()
df

,portfolio,rtns,tickers_pykrx
2008,year 거래소코드 pf_ratio 0 2008 005190 0...,001940 008080 008600 01...,"[005190, 013200, 001600, 014350]"
2009,year 거래소코드 pf_ratio 20 2009 950010 0...,000120 006660 008600 00...,"[950010, 014350]"
2010,year 거래소코드 pf_ratio 32 2010 001940 0...,001940 008600 002410 00...,[950010]
2011,year 거래소코드 pf_ratio 44 2011 950010 0...,009280 013360 025530 00...,"[950010, 900030, 003940]"
2012,year 거래소코드 pf_ratio 64 2012 013360 0...,013360 009280 025530 00...,[003940]
2013,year 거래소코드 pf_ratio 83 2013 001940 0...,001940 006660 025530 00...,[003940]
2014,year 거래소코드 pf_ratio 92 2014 001940 0...,001940 025530 001080 00...,[]
2015,year 거래소코드 pf_ratio 99 2015 001940 ...,001940 025530 004890 00...,[]
2016,year 거래소코드 pf_ratio 106 2016 001940 ...,001940 036530 021820 02...,[]
2017,year 거래소코드 pf_ratio 119 2017 001940 ...,001940 036530 021820 02...,[002300]


In [36]:
# year = df[df['tickers_pykrx'].apply(len) > 0].index
# year

In [37]:
# for idx in year:
#     print(df.loc[idx]['rtns'][df.loc[idx]['tickers_pykrx']])

In [38]:
portfolio_datas.keys()

dict_keys([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [39]:
portfolio_datas[2011].keys()

dict_keys(['portfolio', 'rtns', 'tickers_pykrx'])

In [40]:
total_rtns = []

for year in portfolio_datas.keys():
    datas = portfolio_datas[year]
    pf = datas['portfolio']
    rtns = datas['rtns']
    
    rtns_pf = rtns.copy()
    for col in rtns.columns:
        print(year, col)
        pf_ratio = pf[pf['거래소코드']==col]['pf_ratio'].values[0]
        rtns_pf[col] = rtns[col] * pf_ratio
    rtns_pf['rtn'] = rtns_pf.sum(axis=1)
    rtns_pf = rtns_pf[['rtn']]
    portfolio_datas[year]['pf_rtn'] = rtns_pf
    
    rtns_pf['year'] = year
    total_rtns.append(rtns_pf)
    
total_rtns = pd.concat(total_rtns, axis=0)
total_rtns

2008 001940
2008 008080
2008 008600
2008 012200
2008 084870
2008 008110
2008 005390
2008 002170
2008 004130
2008 023590
2008 006740
2008 092230
2008 002220
2008 006980
2008 008060
2008 004700
2009 000120
2009 006660
2009 008600
2009 006740
2009 003610
2009 002070
2009 077500
2009 008110
2009 012200
2009 023590
2010 001940
2010 008600
2010 002410
2010 006660
2010 003610
2010 006980
2010 077500
2010 009280
2010 092230
2010 013580
2010 023590
2011 009280
2011 013360
2011 025530
2011 001940
2011 006660
2011 003610
2011 014300
2011 092230
2011 009140
2011 002170
2011 009810
2011 002070
2011 002410
2011 029530
2011 008110
2011 009680
2011 004830
2012 013360
2012 009280
2012 025530
2012 002170
2012 002070
2012 036530
2012 009140
2012 092230
2012 003610
2012 006660
2012 001080
2012 013580
2012 010600
2012 005680
2012 069640
2012 024070
2012 012200
2012 029530
2013 001940
2013 006660
2013 025530
2013 001080
2013 002170
2013 009140
2013 005560
2013 005680
2014 001940
2014 025530
2014 001080
2014

,rtn,year
Date,,
2008-12-12,-0.010084,2008
2008-12-15,0.036541,2008
2008-12-16,0.005670,2008
2008-12-17,-0.006704,2008
2008-12-18,0.009727,2008
...,...,...
2023-12-26,-0.000514,2023
2023-12-27,-0.013306,2023
2023-12-28,0.006982,2023


In [41]:
total_rtns

,rtn,year
Date,,
2008-12-12,-0.010084,2008
2008-12-15,0.036541,2008
2008-12-16,0.005670,2008
2008-12-17,-0.006704,2008
2008-12-18,0.009727,2008
...,...,...
2023-12-26,-0.000514,2023
2023-12-27,-0.013306,2023
2023-12-28,0.006982,2023


In [42]:
df

,portfolio,rtns,tickers_pykrx
2008,year 거래소코드 pf_ratio 0 2008 005190 0...,001940 008080 008600 01...,"[005190, 013200, 001600, 014350]"
2009,year 거래소코드 pf_ratio 20 2009 950010 0...,000120 006660 008600 00...,"[950010, 014350]"
2010,year 거래소코드 pf_ratio 32 2010 001940 0...,001940 008600 002410 00...,[950010]
2011,year 거래소코드 pf_ratio 44 2011 950010 0...,009280 013360 025530 00...,"[950010, 900030, 003940]"
2012,year 거래소코드 pf_ratio 64 2012 013360 0...,013360 009280 025530 00...,[003940]
2013,year 거래소코드 pf_ratio 83 2013 001940 0...,001940 006660 025530 00...,[003940]
2014,year 거래소코드 pf_ratio 92 2014 001940 0...,001940 025530 001080 00...,[]
2015,year 거래소코드 pf_ratio 99 2015 001940 ...,001940 025530 004890 00...,[]
2016,year 거래소코드 pf_ratio 106 2016 001940 ...,001940 036530 021820 02...,[]
2017,year 거래소코드 pf_ratio 119 2017 001940 ...,001940 036530 021820 02...,[002300]


In [43]:
total_rtns = pd.merge(
    # total_rtns, df_kospi[['event', '귀속년도', 'rtn']],
    total_rtns, df_kospi[['event', 'rtn']],
    suffixes=('_portfolio', '_kospi'),
    left_index=True, right_index=True,
    how='left'
)
total_rtns

,rtn_portfolio,year,event,rtn_kospi
Date,,,,
2008-12-12,-0.010084,2008,before,-4.383991
2008-12-15,0.036541,2008,before,4.925622
2008-12-16,0.005670,2008,before,0.290981
2008-12-17,-0.006704,2008,before,0.705081
2008-12-18,0.009727,2008,before,0.526611
...,...,...,...,...
2023-12-26,-0.000514,2023,santa,0.118487
2023-12-27,-0.013306,2023,santa,0.419194
2023-12-28,0.006982,2023,santa,1.598624


In [44]:
total_rtns = total_rtns[(total_rtns['event']=='before') | (total_rtns['event']=='santa')]

In [45]:
total_rtns[['year', 'event']].value_counts()

year  event 
2008  before    7
      santa     7
2023  before    7
2022  santa     7
      before    7
2021  santa     7
      before    7
2020  santa     7
      before    7
2019  santa     7
      before    7
2018  santa     7
      before    7
2017  santa     7
      before    7
2016  santa     7
      before    7
2015  santa     7
      before    7
2014  santa     7
      before    7
2013  santa     7
      before    7
2012  santa     7
      before    7
2011  santa     7
      before    7
2010  santa     7
      before    7
2009  santa     7
      before    7
2023  santa     7
Name: count, dtype: int64

파일로 저장

In [46]:
fname2 = fname.replace('portfolio', 'rtns')
print(fname2)

total_rtns.to_csv(fname2)

../datasets/rtns_ncav.csv
